In [ ]:
!pip install './FActScore-main'
!pip install openai==0.27.0

Processing ./drive/MyDrive/Research Works/Hallucination/RAG Prompting/Terminology/eva_term/FActScore-main
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.7 MB/s eta 0:00:00
  Created wheel for factscore: filename=factscore-0.2.0-py3-none-any.whl size=2241213 sha256=5ba6a23e8916bd

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import numpy as np
import pandas as pd
import openai
import os
import subprocess
openai.api_base = "https://api.ai-gaochao.cn/v1"
openai.api_key = ""

In [ ]:
df_all = pd.read_excel('terminology_results.xlsx')

In [ ]:
input_data = []
for i, row in df.iterrows():
    input_data.append({
        "title": row["Term"],
        "text": row["Correct Answer"]
    })

import json
with open("input.jsonl", "w") as f:
    for item in input_data:
        f.write(json.dumps(item) + "\n")


In [ ]:
from factscore.factscorer import FactScorer
import shutil
import os

api_key_content = ''

with open('api.key', 'w') as f:
    f.write(api_key_content)

os.makedirs('.cache/factscore/demos/', exist_ok=True)
shutil.copy('./FActScore-main/factscore/demos/demons.json',
            '.cache/factscore/demos/demons.json')
fs = FactScorer()

name_of_your_knowledge_source = 'test_name'
topics = df["Term"].tolist()

def get_first_5_sentences(text):
    sentences = text.split('.')
    return '.'.join(sentences[:5]) + '.'

generations = df["Generated Explanation"].apply(get_first_5_sentences).tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
fs.register_knowledge_source(name_of_your_knowledge_source,
                             data_path='input.jsonl',
                             db_path='input.db')

input.db is empty. start building DB from input.jsonl...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Finish saving 0M documents (0min)


In [ ]:
import pandas as pd

excel_path = "term_checked.xlsx"

pd.DataFrame(columns=['Topic', 'Score', 'Init Score', 'Respond Ratio', 'Num Facts']).to_excel(excel_path, index=False)

score = 0
init_score = 0
num_facts = 0
respond_ratio = 0

for i in range(len(topics)):
    out = fs.get_score([topics[i]], [generations[i]], knowledge_source=name_of_your_knowledge_source)

    new_row = pd.DataFrame({
        'Topic': [topics[i]],
        'Score': [out["score"]],
        'Init Score': [out["init_score"]],
        'Respond Ratio': [out["respond_ratio"]],
        'Num Facts': [out["num_facts_per_response"]]
    })

    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        book = writer.book
        startrow = writer.sheets['Sheet1'].max_row
        new_row.to_excel(writer, index=False, header=False, startrow=startrow)
    print(f"{i+1} saved")

    score += out["score"] * out["num_facts_per_response"]
    init_score += out["init_score"] * out["num_facts_per_response"]
    respond_ratio += out["respond_ratio"]
    num_facts += out["num_facts_per_response"]

summary = pd.DataFrame({
    'Topic': ['Summary'],
    'Score': [score/num_facts],
    'Init Score': [init_score/num_facts],
    'Respond Ratio': [respond_ratio/len(topics)],
    'Num Facts': [num_facts/len(topics)]
})

with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    book = writer.book
    startrow = writer.sheets['Sheet1'].max_row
    summary.to_excel(writer, index=False, header=False, startrow=startrow)
print("Summary saved")